In [2]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.metrics import mean_squared_error
%matplotlib inline
np.set_printoptions(suppress=True)   
pd.set_option('display.float_format', lambda x: '%.5f' % x) 
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)
import pandas as pd
import re
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
%matplotlib inline
np.set_printoptions(suppress=True)   
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestRegressor
from matplotlib.ticker import FuncFormatter
from matplotlib.pyplot import MultipleLocator
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
import joblib
import scipy.stats
from time import *
from tqdm import tnrange, tqdm_notebook
from time import sleep
from matplotlib.mathtext import _mathtext as mathtext
from sklearn.utils import resample
from sklearn.ensemble import VotingRegressor
mathtext.FontConstantsBase.sup1 = 0.3 

In [3]:
var = locals()

In [443]:
nh3 = pd.read_excel('E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\Global_EF\\df_training_2627_space10fold.xlsx')

In [446]:
len(nh3[nh3['region'] == 'China'])/len(nh3)

0.6783403121431291

In [447]:
len(nh3[nh3['Crop type'] == 'Rice'])/len(nh3),len(nh3[nh3['Crop type'] == 'Wheat'])/len(nh3),len(nh3[nh3['Crop type'] == 'Maize'])/len(nh3)

(0.40159878188047204, 0.2447658926532166, 0.3536353254663114)

In [449]:
nh3['Number of fertilizer applications'] = [1 if x >= 2 else 0 for x in nh3['Number of fertilizer applications']]

In [450]:
# 指定分层列
stratify_columns = ['Fertilizer type', 'Nitrogen placement', 'Number of fertilizer applications', 'Soil tillage practices', 'Crop type']
# 存储bootstrapped数据集的列表
bootstrapped_datasets = []

# 额外生成99个bootstrapped数据集
for i in range(5,104):
    # 对数据集进行分层抽样重采样
    var['nh3_' + str(i)] = resample(nh3, stratify=nh3[stratify_columns], replace=True,random_state = i)
    bootstrapped_datasets.append(var['nh3_' + str(i)])
    print(var['nh3_' + str(i)]['EF'].mean())
    print(var['nh3_' + str(i)]['Number of fertilizer applications'].sum())
    print(len(var['nh3_' + str(i)][var['nh3_' + str(i)]['Crop type'] == 'Rice']))
bootstrapped_datasets.append(nh3)

8.797282112324941
1592
1055
8.875038567786351
1592
1055
8.754782038022457
1592
1055
8.753375695558786
1592
1055
8.847147199894067
1592
1055
8.68843878842965
1592
1055
8.765781158284787
1592
1055
8.98702782225614
1592
1055
8.61854583874271
1592
1055
8.784171803171821
1592
1055
9.015887221112328
1592
1055
8.814722573129265
1592
1055
8.936976769072544
1592
1055
8.74637532319378
1592
1055
8.92977196718132
1592
1055
8.95137319897666
1592
1055
8.81349189928029
1592
1055
8.606396171388216
1592
1055
8.977048350057414
1592
1055
8.592888852744359
1592
1055
8.759541544515178
1592
1055
8.478254497087699
1592
1055
8.604855680659597
1592
1055
8.58300373005396
1592
1055
8.691890413435802
1592
1055
8.79373086412211
1592
1055
8.656556465808949
1592
1055
8.693130602693856
1592
1055
8.94980736999297
1592
1055
8.727905776245759
1592
1055
8.900378642094728
1592
1055
8.767440893127961
1592
1055
8.779207124074864
1592
1055
8.841081972839783
1592
1055
8.68946092789685
1592
1055
8.932947101189107
1592
1055
8.8

In [451]:
len(bootstrapped_datasets)

100

In [465]:
nh3_raw = nh3.rename(columns={'Number of fertilizer applications':'FAT',
                      'Soil tillage practices':'STP',
                      'Average daily air temperature':'Tem',
                      'SOC(g/kg)':'SOC','TN(g/kg)':'TN','BD(g/cm3)':'BD','clay(%)':'Clay','CEC(cmol/kg)':'CEC',
                      'Nitrogen application rate':'Nrate',
                      'Fertilizer type':'Ftype','P+I':'Water','Nitrogen placement':'NP','Crop type':'Ctype'})

In [468]:
for nh3df in bootstrapped_datasets:    
    for i in range(1, len(nh3df['Fertilizer type'].unique()) + 1) :
        nh3df[nh3df['Fertilizer type'].unique()[i - 1]] = np.array(nh3df['Fertilizer type'] == nh3df['Fertilizer type'].unique()[i - 1]).astype(np.int32)
        i += 1
    del nh3df['Fertilizer type']

In [469]:
for nh3df in bootstrapped_datasets:   
    for i in range(1, len(nh3df['Nitrogen placement'].unique()) + 1) :
        nh3df[nh3df['Nitrogen placement'].unique()[i - 1]] = np.array(nh3df['Nitrogen placement'] == nh3df['Nitrogen placement'].unique()[i - 1]).astype(np.int32)
        i += 1
    del nh3df['Nitrogen placement']

In [470]:
for nh3df in bootstrapped_datasets:   
    nh3df['Soil tillage practices'] = [1 if x=='NT' else 0 for x in nh3df['Soil tillage practices']]

In [471]:
for nh3df in bootstrapped_datasets:   
    for i in range(1, len(nh3df['Crop type'].unique()) + 1) :
        nh3df[nh3df['Crop type'].unique()[i - 1]] = np.array(nh3df['Crop type'] == nh3df['Crop type'].unique()[i - 1]).astype(np.int32)
        i += 1
    del nh3df['Crop type']

In [472]:
nh3.describe()

,Longitude,Latitude,Number of fertilizer applications,Soil tillage practices,P+I,Average daily air temperature,SOC(g/kg),TN(g/kg),pH,BD(g/cm3),clay(%),CEC(cmol/kg),Nitrogen application rate,EF,U,others,CRF,manure,AN,SBC,Mix,DPM,Rice,Wheat,Maize
count,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000,2627.00000
mean,76.71030,28.67909,0.60601,0.10773,782.49759,20.51783,14.88120,1.78928,6.92639,1.31897,25.69515,15.20493,178.16510,8.75839,0.50285,0.10354,0.24210,0.13247,0.01903,0.45984,0.21203,0.32813,0.40160,0.24477,0.35364
std,74.53126,18.80983,0.48872,0.31009,456.30313,6.86699,14.78764,3.02887,1.09143,0.13506,13.56517,8.70027,77.56544,7.54155,0.50009,0.30472,0.42844,0.33907,0.13667,0.49848,0.40882,0.46962,0.49031,0.43003,0.47819
min,-119.50000,-43.63333,0.00000,0.00000,13.84363,-4.50000,0.65000,0.01190,4.60000,0.66000,1.00000,0.05000,26.70000,0.00513,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,85.91667,28.97361,0.00000,0.00000,368.20000,16.61000,7.47620,0.85000,6.00000,1.26000,16.70000,11.00000,120.00000,2.66253,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
50%,113.55000,31.94500,1.00000,0.00000,759.00000,22.83333,11.50000,1.21499,7.15000,1.33000,21.86000,13.66667,180.00000,6.62000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
75%,117.86667,36.83000,1.00000,0.00000,1129.34615,25.60000,18.50000,1.94000,7.90000,1.40000,31.50000,17.70000,225.00000,13.22009,1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,1.00000,1.00000,0.00000,1.00000
max,172.46667,56.45000,1.00000,1.00000,2867.54615,35.00000,145.00000,31.00000,9.00000,1.94000,72.10000,95.90000,624.00000,31.91742,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000


In [476]:
for nh3df in bootstrapped_datasets[:-1]: 
    nh3df = sklearn.utils.shuffle(nh3df,random_state=2022)  
    nh3df.index = range(0,len(nh3df))
    nh3df

In [479]:
nh3 = sklearn.utils.shuffle(nh3,random_state=2022) 
nh3.index = range(0,len(nh3))
nh3

,Longitude,Latitude,Number of fertilizer applications,Soil tillage practices,P+I,Average daily air temperature,SOC(g/kg),TN(g/kg),pH,BD(g/cm3),clay(%),CEC(cmol/kg),Nitrogen application rate,EF,region,U,others,CRF,manure,AN,SBC,Mix,DPM,Rice,Wheat,Maize
0,-109.40000,48.30000,1,1,263.88196,-4.50000,18.50000,1.67000,6.30000,1.24500,22.00000,22.00000,100.00000,16.30000,NonChina,1,0,0,0,0,1,0,0,0,1,0
1,-91.26667,33.75000,0,0,971.54615,25.00000,16.40000,2.38000,6.60000,1.34500,30.25000,18.50000,168.00000,7.41176,NonChina,0,1,0,0,0,1,0,0,1,0,0
2,120.67000,30.83000,1,0,938.20000,28.35000,20.30162,2.65000,6.78000,1.33000,51.70000,8.10000,180.00000,6.00000,China,1,0,0,0,0,1,0,0,1,0,0
3,85.88333,20.50000,0,0,2174.24615,27.14286,7.10000,0.72500,6.10000,1.49000,18.75000,9.75000,60.00000,1.22000,NonChina,0,0,1,0,0,0,0,1,1,0,0
4,124.33000,43.30000,0,0,899.81251,20.18000,10.96200,1.00000,5.80000,1.50000,1.00000,23.80000,224.00000,3.95982,China,0,0,1,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622,115.78333,38.13333,1,0,206.56090,19.94452,11.19490,1.00000,8.60000,1.39000,18.00000,14.00000,180.00000,7.16667,China,1,0,0,0,0,1,0,0,0,0,1
2623,112.50000,28.51000,1,0,2867.54615,24.00000,20.78140,2.05000,5.16000,1.41000,41.75000,11.00000,150.00000,2.54000,China,0,0,0,1,0,0,1,0,1,0,0
2624,111.50000,28.91667,1,0,1195.90000,24.00000,14.10000,1.78000,5.38000,1.21000,28.20000,11.40000,81.30000,12.66913,China,1,0,0,0,0,0,1,0,1,0,0
2625,119.16869,31.94500,1,0,904.76117,9.85000,9.90000,0.68000,6.30000,1.41000,21.00000,11.00000,100.00000,7.60000,China,0,0,0,1,0,0,1,0,0,1,0


In [480]:
#data transformation
EFraw = nh3['EF'].copy()
#Box-Cox transformation
pt = preprocessing.PowerTransformer(method='box-cox', standardize=False)
nh3['EF'] = pt.fit_transform(nh3['EF'].ravel().reshape(-1, 1))

In [481]:
for nh3df in bootstrapped_datasets[:-1]: 
    nh3df['EF'] = pt.transform(nh3df['EF'].ravel().reshape(-1, 1))

In [483]:
pt.inverse_transform(np.array([[1]]))  
pt.lambdas_  

array([0.33617797])

In [484]:
nh3['EF'].mean()

2.5919913061213378

In [486]:
X_col_ori = list(nh3.columns)

In [487]:
X2 = nh3[X_col_ori] 
X_col_ori.remove('EF')
X_col_ori.remove('region')
X_col_ori.remove('Longitude')
X_col_ori.remove('Latitude')

In [488]:
X = nh3[X_col_ori]

In [489]:
for i in range(5,104):
    var['X_'+str(i)] = var['nh3_' + str(i)][X_col_ori]
    var['X_'+str(i)] = var['X_'+str(i)].rename(columns={'Number of fertilizer applications':'FAT',
                      'Soil tillage practices':'STP',
                      'Average daily air temperature':'Tem',
                      'SOC(g/kg)':'SOC','TN(g/kg)':'TN','BD(g/cm3)':'BD','clay(%)':'Clay','CEC(cmol/kg)':'CEC',
                      'Nitrogen application rate':'Nrate',
                      'U':'UOA','P+I':'Water'})

In [490]:
X2 = X2.rename(columns={'Number of fertilizer applications':'FAT',
                      'Soil tillage practices':'STP',
                      'Average daily air temperature':'Tem',
                      'SOC(g/kg)':'SOC','TN(g/kg)':'TN','BD(g/cm3)':'BD','clay(%)':'Clay','CEC(cmol/kg)':'CEC',
                      'Nitrogen application rate':'Nrate',
                      'U':'UOA','P+I':'Water'})

In [491]:
X = X.rename(columns={'Number of fertilizer applications':'FAT',
                      'Soil tillage practices':'STP',
                      'Average daily air temperature':'Tem',
                      'SOC(g/kg)':'SOC','TN(g/kg)':'TN','BD(g/cm3)':'BD','clay(%)':'Clay','CEC(cmol/kg)':'CEC',
                      'Nitrogen application rate':'Nrate',
                      'U':'UOA','P+I':'Water'})

In [492]:
X_col = list(X.columns)

In [495]:
Y = nh3['EF'].values.reshape(len(nh3),1)

In [496]:
for i in range(5,104):
    var['Y_'+str(i)] = var['nh3_' + str(i)]['EF'].values.reshape(len(var['nh3_' + str(i)]),1)

In [497]:
X.shape, Y.shape

((2627, 22), (2627, 1))

In [498]:
var['X_'+str(i)].shape, var['Y_'+str(i)].shape

((2627, 22), (2627, 1))

In [335]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 266) 

In [229]:
x_train.shape, x_test.shape

((2101, 22), (526, 22))

In [230]:
y_train.shape, y_test.shape

((2101, 1), (526, 1))

In [231]:
n_estimators = range(100,101)
random_state = range(2023,2024)
max_features= range(1,22)
min_samples_leaf = range(1,5)
max_depth = [3, 5, 8, 10, 15, 20, 25, 30, 35, 40, None] 

In [232]:
param_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
             'random_state': random_state,
             'min_samples_leaf': min_samples_leaf,
             'max_depth': max_depth}

In [233]:
grid = GridSearchCV(RandomForestRegressor(), param_grid, cv=10,n_jobs = -1,return_train_score=False)

In [234]:
begin_time = time()

In [235]:
grid.fit(x_train, y_train.ravel())

GridSearchCV(cv=10, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 8, 10, 15, 20, 25, 30, 35, 40,
                                       None],
                         'max_features': range(1, 22),
                         'min_samples_leaf': range(1, 5),
                         'n_estimators': range(100, 101),
                         'random_state': range(2023, 2024)})

In [236]:
end_time = time()
run_time = end_time-begin_time
print ('time cost：',run_time/60, 'min') #该循环程序运行时间： 1.4201874732

time cost： 5.562656692663828 min


In [237]:
grid.best_params_

{'max_depth': 30,
 'max_features': 8,
 'min_samples_leaf': 1,
 'n_estimators': 100,
 'random_state': 2023}

In [238]:
n_estimators = range(500,501)
random_state = range(2023,2024)
max_features= range(6,16)
min_samples_leaf = range(1,2)
max_depth = [15, 20, 25, 30, 35, 40, None]

In [239]:
param_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
             'random_state': random_state,
             'min_samples_leaf': min_samples_leaf,
             'max_depth': max_depth}

In [240]:
grid = GridSearchCV(RandomForestRegressor(), param_grid, cv=10,n_jobs = 7,return_train_score=False)

In [241]:
begin_time = time()

In [242]:
grid.fit(x_train, y_train.ravel())

GridSearchCV(cv=10, estimator=RandomForestRegressor(), n_jobs=7,
             param_grid={'max_depth': [15, 20, 25, 30, 35, 40, None],
                         'max_features': range(6, 16),
                         'min_samples_leaf': range(1, 2),
                         'n_estimators': range(500, 501),
                         'random_state': range(2023, 2024)})

In [243]:
end_time = time()
run_time = end_time-begin_time
print ('time cost：',run_time/60, 'min')

time cost： 4.449661755561829 min


In [244]:
grid.best_params_

{'max_depth': 25,
 'max_features': 14,
 'min_samples_leaf': 1,
 'n_estimators': 500,
 'random_state': 2023}

In [245]:
grid.score(x_test, y_test)

0.7942672058366329

In [246]:
grid.predict(np.array(x_test.iloc[10]).reshape(1, -1)),y_test[10]

E:\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


(array([4.94927903]), array([6.23181828]))

In [247]:
grid.best_score_

0.6637056344500083

In [248]:
grid.score(x_train, y_train)

0.9201557151144562

In [249]:
np.sqrt(mean_squared_error(grid.predict(x_test), y_test))

0.8768369156201443

In [250]:
np.sqrt(mean_squared_error(grid.predict(X), Y))

0.632128603314749

In [251]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_min_samples_leaf,param_n_estimators,param_random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,1.86059,0.04861,0.06690,0.00523,15,6,1,500,2023,"{'max_depth': 15, 'max_features': 6, 'min_samples_leaf': 1, 'n_estimators': 500, 'random_state':...",0.57398,0.70660,0.73437,0.63544,0.65798,0.58306,0.69311,0.65087,0.65326,0.69867,0.65873,0.04933,70
1,2.00984,0.04704,0.05722,0.00604,15,7,1,500,2023,"{'max_depth': 15, 'max_features': 7, 'min_samples_leaf': 1, 'n_estimators': 500, 'random_state':...",0.57326,0.71163,0.74195,0.63444,0.65983,0.58604,0.69079,0.65010,0.65496,0.70437,0.66074,0.05105,64
2,2.03981,0.04350,0.05486,0.00554,15,8,1,500,2023,"{'max_depth': 15, 'max_features': 8, 'min_samples_leaf': 1, 'n_estimators': 500, 'random_state':...",0.56603,0.70716,0.73797,0.63836,0.66237,0.58877,0.69416,0.65057,0.65681,0.70040,0.66026,0.05058,68
3,2.16403,0.04320,0.05424,0.00424,15,9,1,500,2023,"{'max_depth': 15, 'max_features': 9, 'min_samples_leaf': 1, 'n_estimators': 500, 'random_state':...",0.56531,0.71246,0.74306,0.63827,0.65685,0.58369,0.69480,0.65037,0.65339,0.70584,0.66040,0.05326,67
4,2.40852,0.06996,0.05629,0.00671,15,10,1,500,2023,"{'max_depth': 15, 'max_features': 10, 'min_samples_leaf': 1, 'n_estimators': 500, 'random_state'...",0.56910,0.71253,0.74298,0.63597,0.65842,0.58559,0.69526,0.64972,0.65508,0.70503,0.66097,0.05236,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2.57103,0.03751,0.05812,0.00538,None,11,1,500,2023,"{'max_depth': None, 'max_features': 11, 'min_samples_leaf': 1, 'n_estimators': 500, 'random_stat...",0.56967,0.71390,0.74371,0.64121,0.66245,0.58671,0.69598,0.64850,0.64994,0.70988,0.66219,0.05270,27
66,2.69741,0.04286,0.05523,0.00372,None,12,1,500,2023,"{'max_depth': None, 'max_features': 12, 'min_samples_leaf': 1, 'n_estimators': 500, 'random_stat...",0.56595,0.71099,0.74640,0.64143,0.65976,0.58094,0.69466,0.65029,0.65150,0.71009,0.66120,0.05419,50
67,2.86249,0.03899,0.05624,0.00447,None,13,1,500,2023,"{'max_depth': None, 'max_features': 13, 'min_samples_leaf': 1, 'n_estimators': 500, 'random_stat...",0.57008,0.71136,0.74377,0.64798,0.66322,0.58511,0.70216,0.65199,0.65163,0.70634,0.66336,0.05239,4
68,3.03904,0.06599,0.05333,0.00606,None,14,1,500,2023,"{'max_depth': None, 'max_features': 14, 'min_samples_leaf': 1, 'n_estimators': 500, 'random_stat...",0.56707,0.70802,0.74604,0.64631,0.66392,0.58395,0.69731,0.65538,0.65479,0.70864,0.66314,0.05295,8


In [252]:
pd.DataFrame(grid.cv_results_).iloc[grid.best_index_,:]

mean_fit_time                                                                                                         3.03095
std_fit_time                                                                                                          0.02483
mean_score_time                                                                                                       0.05471
std_score_time                                                                                                        0.00422
param_max_depth                                                                                                            25
param_max_features                                                                                                         14
param_min_samples_leaf                                                                                                      1
param_n_estimators                                                                                                    

In [253]:
RF = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'], max_features=grid.best_params_['max_features'],min_samples_leaf = grid.best_params_['min_samples_leaf'],max_depth = grid.best_params_['max_depth'],oob_score = True,random_state=grid.best_params_['random_state'],n_jobs=14)#random_state=grid.best_params_['random_state']   #最佳随机状态 = 112
RF.fit(x_train, y_train.ravel())

RandomForestRegressor(max_depth=25, max_features=14, n_estimators=500,
                      n_jobs=14, oob_score=True, random_state=2023)

In [254]:
RF

RandomForestRegressor(max_depth=25, max_features=14, n_estimators=500,
                      n_jobs=14, oob_score=True, random_state=2023)

In [255]:
joblib.dump(RF,'E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\RF_V6_alldatatraining_trainingset.pkl') 

['E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\RF_V6_alldatatraining_trainingset.pkl']

In [256]:
RF.score(x_test, y_test)

0.7942672058366329

In [220]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import max_error
from quantile_forest import RandomForestQuantileRegressor

In [221]:
sdf = []  #100个重采样数据集训练
for i in tqdm_notebook(range(5,104)):
    RF = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'], max_features=grid.best_params_['max_features'],min_samples_leaf = grid.best_params_['min_samples_leaf'],max_depth = grid.best_params_['max_depth'],oob_score = False,random_state=grid.best_params_['random_state'],n_jobs=14)#random_state=grid.best_params_['random_state']   #最佳随机状态 = 2023
    RF.fit(var['X_'+str(i)], var['Y_'+str(i)].ravel())
    joblib.dump(RF,'E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\RF_V6_alldatatraining_100bootstrap_'+ str(i) +'.pkl')
    sdf.append(pt.inverse_transform(np.array(RF.predict(x_test).reshape(-1, 1)))[0])
print(np.mean(sdf))
print(np.std(sdf))
print(np.std(sdf)/np.mean(sdf))
print(np.percentile(sdf, 2.5 ))
print(np.percentile(sdf, 97.5 ))

<ipython-input-221-b896a9487e6d>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(5,104)):


  0%|          | 0/99 [00:00<?, ?it/s]

6.323512123926963
0.729646650182338
0.11538629734281593
5.458291628884666
8.006867100276251


In [222]:
RF = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'], max_features=grid.best_params_['max_features'],min_samples_leaf = grid.best_params_['min_samples_leaf'],max_depth = grid.best_params_['max_depth'],oob_score = True,random_state=grid.best_params_['random_state'],n_jobs=14)
RF.fit(X, Y.ravel())

RandomForestRegressor(max_depth=25, max_features=14, n_estimators=500,
                      n_jobs=14, oob_score=True, random_state=2023)

In [ ]:
joblib.dump(RF,'E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\RF_V6_alldatatraining_100bootstrap_'+ str(104) +'.pkl')

In [197]:
##save model
joblib.dump(RF,'E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\RF_V6_alldatatraining.pkl')
joblib.dump(grid,'E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\RF_grid_V6_alldatatraining.pkl')
joblib.dump(pt,'E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\boxcox_V6_alldatatraining.pkl')

['E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\boxcox_V6_alldatatraining.pkl']